# Creating a synthetic Q&A dataset

In [1]:
import os

proxy = 'http://dell-1.star:7890' # 3090 docker
# proxy = 'http://127.0.0.1:7890' # clash
# proxy = 'http://127.0.0.1:1080' # naiveproxy

os.environ['http_proxy'] = proxy 
os.environ['HTTP_PROXY'] = proxy
os.environ['https_proxy'] = proxy
os.environ['HTTPS_PROXY'] = proxy

## Read in data, and create a context

In [2]:
import pandas as pd
df = pd.read_csv("docx/qa/total.csv")
df['context'] = "summary: " +  df.summary + "\n" +  "content: " + df.content
df.head()

,summary,content,summary_token_num,content_token_num,token_num,context
0,简介,中国联通手机营业厅分为网页版和客户端，提供7×24小时服务。目前功能包括：话费查询、交费充值...,1,69,70,summary: 简介\ncontent: 中国联通手机营业厅分为网页版和客户端，提供7×2...
1,访问方式#网页版,中国联通手机营业厅网页版wap.10010.com是面向普通手机上网用户、可提供7×24小时...,6,60,66,summary: 访问方式#网页版\ncontent: 中国联通手机营业厅网页版wap.10...
2,访问方式#客户端,下载方式：手机登陆wap.10010.com下载手机登录www.10010.com下载目前，...,5,96,101,summary: 访问方式#客户端\ncontent: 下载方式：手机登陆wap.10010...
3,常用功能#客户端各功能,当月话费用户登录客户端后，点击当月话费按键，可显示用户当月话费情况套餐余量用户登录客户端后，...,7,312,319,summary: 常用功能#客户端各功能\ncontent: 当月话费用户登录客户端后，点击...
4,常用功能#手机营业厅网页版功能列表,网页版手机营业厅wap.10010.com常用功能1、话费查询话费查询目前可提供当月话费、可...,10,1110,1120,summary: 常用功能#手机营业厅网页版功能列表\ncontent: 网页版手机营业厅w...


## Create questions based on the context

In [3]:
import openai
openai.api_key_path = ".openai-key"

In [5]:
import requests

# requests.get("http://google.com")

context = """
summary: 基础知识#业务介绍#后付费用户定义
content: 后付费用户须签约和登记用户资料；先使用，后付费，按月缴费，享受客户俱乐部、积分等客户服务。可对后付费用户进行信用额度操纵
"""
response = openai.Completion.create(
            engine="davinci-instruct-beta-v3",
            prompt=f"请根据以下文本生成问题，尽可能使用简体中文\n\n文本: {context}\n\n问题:\n1.",
            temperature=0,
            max_tokens=257,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
            stop=["\n\n"]
        )
print(response.choices[0].text)

后付费用户是指什么?
2.后付费用户必须签约和登记用户资料吗?
3.后付费用户先使用，后付费，按月缴费，享受客户俱乐部、积分等客户服务。对后付费用户进行信用额度操纵可以吗?


In [19]:

def get_questions(context):
    try:
        response = openai.Completion.create(
            engine="davinci-instruct-beta-v3",
            prompt=f"请根据以下文本生成问题,尽可能使用简体中文\n\n文本: {context}\n\n问题:\n1.",
            temperature=0,
            max_tokens=257,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
            stop=["\n\n"]
        )
        return response['choices'][0]['text']
    except:
        return ""

df['questions']= df.context.apply(get_questions)
df['questions'] = "1." + df.questions

In [20]:
print(df[['questions']].values)

[['1. 中国联通手机营业厅分为网页版和客户端吗?\n2. 客户端提供7×24小时服务吗?\n3. 目前功能包括哪些?\n4. 业务查询功能包括哪些?\n5. 积分查询功能包括哪些?\n6. 密码修改功能包括哪些?\n7. 网点查询功能包括哪些?\n8. 手机号码归属地功能包括哪些?\n9. 3G网络']
 ['1. 中国联通手机营业厅网页版wap.10010.com是面向普通手机上网用户、可提供7×24小时服务的营销平台吗?\n2. 您只需在手机浏览器上输入wap.10010.com，凭手机号码和服务密码，即可轻松访问中国联通手机营业厅吗?']
 ['1. 下载手机登录www.10010.com下载的用户有哪些?\n2. 不同手机品牌、不同用户类型（后付费/预付费）、不同网络用户（2G/3G用户）看到的版本和可使用的功能会有哪些差别?']
 ['1. 当月话费用户登录客户端后，点击当月话费按键，可显示用户当月话费情况套餐余量用户登录客户端后，点击套餐余量按键，可显示用户当月套餐使用情况历史账单用户登录客户端后，点击历史账单按键，可显示用户历史账单使用情况通话详单用户登录客户']
 ['1.']
 ['1. 中国联通电子渠道的部分功能需要您输入手机号码和服务密码，如您忘记服务密码，可通过以下方式重新设置。请问，重新设置密码的方式有哪些？\n2. 通过手机营业厅在手机浏览器上输入wap.10010.com登录中国联通手机营业厅—>进入登录页面—>点击“忘记密码”—>']
 ['1. 什么是网络的定义?\n2. 中国联通采用的是什么标准?']
 ['1. 与其它标准相比,全球开通网络最多是什么样的?\n2. 全球有多少个国家具有网络覆盖全球市场?\n3. 全球市场占有率最高的终端款式是什么?\n4. 支持商用终端的款式数量最多是什么?']
 ['1. 开通使用中国联通标准资费的用户是否需要差不多套餐?\n2. 开通使用中国联通标准资费的用户是否需要无线上网卡?\n3. 开通使用中国联通标准资费的用户是否需要套餐?']
 ['1.后付费用户须签约和登记用户资料；先使用，后付费，按月缴费，享受客户俱乐部、积分等客户服务。可对后付费用户进行信用额度操纵']
 ['1. 什么是标准产品〔预付费〕?\n2. 先付费后使用的产品是什么?']
 ['1. 使用标准产品〔预付费〕的用

## Create answers based on the context

In [21]:
def get_answers(row):
    try:
        response = openai.Completion.create(
            engine="davinci-instruct-beta-v3",
            prompt=f"根据以下文本生成问题的答案,尽可能使用简体中文\n\n文本: {row.context}\n\n问题:\n{row.questions}\n\n答案:\n1.",
            temperature=0,
            max_tokens=257,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0
        )
        return response['choices'][0]['text']
    except Exception as e:
        print (e)
        return ""


df['answers']= df.apply(get_answers, axis=1)
df['answers'] = "1." + df.answers

This model's maximum context length is 2049 tokens, however you requested 4423 tokens (4166 in your prompt; 257 for the completion). Please reduce your prompt; or completion length.
This model's maximum context length is 2049 tokens, however you requested 4525 tokens (4268 in your prompt; 257 for the completion). Please reduce your prompt; or completion length.
This model's maximum context length is 2049 tokens, however you requested 2265 tokens (2008 in your prompt; 257 for the completion). Please reduce your prompt; or completion length.
This model's maximum context length is 2049 tokens, however you requested 2066 tokens (1809 in your prompt; 257 for the completion). Please reduce your prompt; or completion length.
This model's maximum context length is 2049 tokens, however you requested 2253 tokens (1996 in your prompt; 257 for the completion). Please reduce your prompt; or completion length.
This model's maximum context length is 2049 tokens, however you requested 2151 tokens (189

In [22]:
print(df[['answers']].values)

[['1. 中国联通手机营业厅分为网页版和客户端。\n2. 客户端提供7×24小时服务。\n3. 目前功能包括话费查询、交费充值、3G套餐变更、业务查询、积分查询、密码修改、积分兑换、网点查询、手机号码归属地、3G网络覆盖查询、PUK码查询、3G专区、促销活动等。\n4. 业务查询']
 ['1. 中国联通手机营业厅网页版wap.10010.com是面向普通手机上网用户、可提供7×24小时服务的营销平台。\n2. 您只需在手机浏览器上输入wap.10010.com，凭手机号码和服务密码，即可轻松访问中国联通手机营业厅。']
 ['1. 下载手机登录www.10010.com下载的用户有iPhone版、iPad版、Android版、乐Phone版及Symbian版共五个版本的客户端。\n2. 不同手机品牌、不同用户类型（后付费/预付费）、不同网络用户（2G/3G用户）看到的版本和可使用的功能会有所差别。']
 ['1. 当月话费用户登录客户端后，点击当月话费按键，可显示用户当月话费情况套餐余量用户登录客户端后，点击套餐余量按键，可显示用户当月套餐使用情况历史账单用户登录客户端后，点击历史账单按键，可显示用户历史账单使用情况通话详单用户登录客户']
 ['1.']
 ['1. 中国联通电子渠道的部分功能需要您输入手机号码和服务密码，如您忘记服务密码，可通过以下方式重新设置。请问，重新设置密码的方式有哪些？\n2. 通过手机营业厅在手机浏览器上输入wap.10010.com登录中国联通手机营业厅—>进入登录页面—>点击“忘记密码”—>']
 ['1. 网络的定义是指第三代移动通信技术，是将无线通信与互联网等多媒体通信结合的新一代移动通信系统。\n2. 中国联通采用的是通信标准。']
 ['1. 全球开通网络最多是采用制式的。\n2. 全球有个国家具有网络覆盖全球市场。\n3. 全球市场占有率最高的终端款式是支持商用终端的款式。\n4. 支持商用终端的款式数量最多是款。']
 ['1. 不需要;\n2. 不需要;\n3. 不需要.']
 ['1.后付费用户需要签约和登记用户信息；先使用，后付费，按月付费，享受客户俱乐部、积分等客户服务。可以对后付费用户进行信用额度操纵']
 ['1. 标准产品〔预付费〕是实时扣费、先付费后使用的产品。\n2. 先付费后使用的产品是实时扣费、

In [23]:
df = df.dropna().reset_index().drop('index',axis=1)
df.to_csv("docx/qa/qa-chinese.csv", index=False)

## Debug proxy 

In [16]:
test_context = df.context[0]
response = openai.Completion.create(
            engine="davinci-instruct-beta-v3",
            prompt=f"请根据以下文本生成问题\n\n文本: {test_context}\n\n问题:\n1.",
            temperature=0,
            max_tokens=800,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
            stop=["\n\n"]
        )


In [17]:
print(response['choices'][0]['text'])

 中国联通采用的是什么标准?
2. 网络的定义是指第三代移动通信技术，是将无线通信与互联网等多媒体通信结合的新一代移动通信系统，能够提供包括可视   、无线上网，手机上网、手机电视、手机音乐等多种信息服务。

